In [1]:
using HDF5
using Plots
using DelimitedFiles

In [2]:
# fid = h5open(filename, mode)

In [3]:
function bin(z, bins)  # bins must be increasing
    min_bin, min_i = findmin(bins)
    max_bin, max_i = findmax(bins)
    if z < min_bin
        return min_i
    elseif z > max_bin
        return max_i
    end
    
    for (i, b) in enumerate(bins)
        if z < b
            return i
        end
    end
    
    return length(bins)
end

bin (generic function with 1 method)

In [4]:
function sum_in_bin(redshifts, fluxes, bins)
    results = zeros(length(bins))
    for (z, f) in zip(redshifts, fluxes)
        i = bin(z, bins)
        results[i] += f
    end
    return results
end

sum_in_bin (generic function with 1 method)

In [5]:
z_bins = collect(0.0:0.1:5.0)
z_mids = (z_bins) .+ 0.1 / 2;

# freq = "353"

In [6]:
function genbin(redshiftname, z_bins, freq)
    fluxname = replace(redshiftname, ".h5"=>"_flux_$(freq).h5")
    redshifts =  read(h5open(redshiftname, "r"), "redshift")
    fluxes = read(h5open(fluxname, "r"), "flux")
    return sum_in_bin(redshifts, fluxes, z_bins)
end

genbin (generic function with 1 method)

In [7]:
sourcedir = "/global/cscratch1/sd/xzackli/cib_sources/sources/"


function threadf(files, z_bins, freq)
    xf = [[] for f in files]
    Threads.@threads for i in 1:length(files)
        f = files[i]
        xf[i] = genbin(f, z_bins, freq)
    end
    return xf
end


threadf (generic function with 1 method)

In [8]:
for freq in ["143", "217", "353", "545"]
    results = threadf(
        [
            "/global/cscratch1/sd/xzackli/cib_sources/sources/" * s for s in 
            ["cen_chunk1.h5", "cen_chunk2.h5", "sat_chunk1.h5", "sat_chunk2.h5"]
        ], z_bins, freq
    )
    flux_sum = sum(results)
    open("$(freq)_source_counts.txt", "w") do io
       write(io, "# z_bin_left flux_sum\n")
       writedlm(io, [z_bins flux_sum])
    end
end